# Выпускной проект. Анализ поведения пользователей в мобильном приложении


## Описание проекта

Я работаю в команде сервиса «Ненужные вещи». В мобильном приложении этого сервиса пользователи продают свои ненужные вещи, размещая их на доске объявлений. Нужно разобраться, как ведут себя пользователи мобильного приложения. 

## Цель проекта

Управление вовлеченностью клиентов (адаптация приложения по целевой и смежной аудитории) на основе данных о поведении пользователей.
Получить на основе поведения пользователей гипотезы о том как можно было бы улучшить приложение с точки зрения пользовательского опыта.

Дать ответы на следующие вопросы:
* Проанализировать связь целевого события — просмотра контактов — и других действий пользователей. 
* Оценить, какие действия чаще совершают те пользователи, которые просматривают контакты. 

## Описание данных

В датасетах содержатся данные пользователей, впервые совершивших действия в приложении после 7 октября 2019 года. 

Колонки в  /datasets/mobile_sources.csv : 
* userId  — идентификатор пользователя,
* source  — источник, с которого пользователь установил приложение.

Колонки в  /datasets/mobile_dataset.csv : 
* event.time  — время совершения, 
* user.id  — идентификатор пользователя,
* event.name  — действие пользователя. Виды действий:
    * advert_open  — открыл карточки объявления,
    * photos_show  — просмотрел фотографий в объявлении,
    * tips_show  — увидел рекомендованные объявления,
    * tips_click  — кликнул по рекомендованному объявлению,
    * contacts_show  и  show_contacts  — посмотрел номер телефона,
    * contacts_call  — позвонил по номеру из объявления,
    * map  — открыл карту объявлений,
    * search_1 — search_7  — разные действия, связанные с поиском по сайту,
    * favorites_add  — добавил объявление в избранное.

<!--- TODO:  --->

## Декомпозиция этапов

- [x] Описание проекта, его цели и данных 
- [x] Загрузка данных
- [x] Первичный осмотр данных с помощью библиотеки pandas-profiling
    * Проверка на пропуски/дубликаты
    * Проверка на наличие корреляций
    * Просмотр статистики категориальных и количественных данных
    * Выявление выбросов и аномалий
- [x] Предобработка данных:
    * Обработка пропусков/дубликатов
    * При необходимости переименование столбцов
    * При необходимости удаление столбцов и создание новых столбцов-признаков
- [x] Проведение когортного анализа для выявления как поведение пользователей изменяется во времени
    * Создание датафрейма sessions для анализа сессий пользователей
        * Расчет тайм-аута сессии с помощью диаграммы размаха - просмотр аномальных значений. В качестве тайм-аута берется минимальное аномальное значение времени сессии
        * Создание датафрейма sessions с выделением сессий пользователей по тайм-аутам
        * Удаление повторяющихся событий из сессий пользователей
    * Визуализация сессий с помощью диаграммы Санкея по сессиям *session_id* как пользователи проходят разные этапы в нашем приложении
    * Проанализировать связь целевого события (просмотра контактов) и других действий пользователей 
        * Анализ сценариев по диаграмме Санкея до совершения просмотра контактов
        * Построение воронки по основным сценариям в разрезе уникальных пользователей
    * Оценить, какие действия чаще совершают те пользователи, которые просматривают контакты
        * Рассчитать относительную частоту событий в разрезе двух групп пользователей: 
            * группа пользователей, которые смотрели контакты *contacts_show*
            * группа пользователей, которые не смотрели контакты *contacts_show*
    * Расчет метрик и их визуализация
        * Рассчитать conversion и отобразить диаграмму
        * Рассчитать retention и отобразить диаграмму
        * Возможно еще какие-нибудь метрики
- [x] Проверить статистические гипотезы с помощью AB тестов:
    *  Одни пользователи совершают действия  tips_show  и  tips_click , другие — только  tips_show . Проверить гипотезу: конверсия в просмотры контактов различается у этих двух групп.
        * Нулевая гипотеза: Нет статистически значимой разницы в конверсии в просмотры контактов у этих двух групп
        * Альтернативная: Присутствует статистически значимая разница в конверсии в просмотры контактов у этих двух групп        
    *  Проверить собственную гипотезу. Пользователи одной группы совершают действие advert_open, другой любое действие вида search_* 
        * Нулевая гипотеза: Нет статистически значимой разницы в конверсии в просмотры контактов у этих двух групп
        * Альтернативная: Присутствует статистически значимая разница в конверсии в просмотры контактов у этих двух групп        
- [x] Оценить эффективность функционала и выявить поведенческие паттерны проанализировав сценарии и продуктовые метрики
- [x] Сделать общие выводы и рекомендации для улучшения продуктовых метрик приложения
- [x] По итогам исследования подготовить презентацию в формате pdf, прикрепив ссылку на файл в проекте.



# Загрузка библиотек и глобальных переменных

In [1]:
# ====================================================== Установка библиотек ======================================================
# %pip install --quiet ydata-profiling
# %pip install --quiet duckdb

In [2]:
# ======================================================== Импорты и настройки =====================================================================

import pandas as pd
import numpy as np
import warnings
import duckdb

import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from ydata_profiling import ProfileReport
from statsmodels. stats.proportion import proportions_ztest

import requests
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{: .2f}'.format # отображение 2 знаков после запятой в float
pd.options.display.max_rows = 50
pd.options.display.max_columns = False
pd.options.display.max_colwidth = False
pd.options.display.precision = 2


# Пользовательские функции

Ф-ции для работы с диаграммой Санкей:

In [3]:
# ============================================== Ф-ции для работы с диаграммой Санкей ==============================================================

def add_features(df):
    
    """Функция генерации новых столбцов для исходной таблицы

    Args:
        df (pd.DataFrame): исходная таблица.
    Returns:
        pd.DataFrame: таблица с новыми признаками.
    """
    
    # сортируем по id и времени
    sorted_df = df.sort_values(by=['session_id', 'event_time']).copy()
    # добавляем шаги событий
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    
    # добавляем узлы-источники и целевые узлы
    # узлы-источники - это сами события
    sorted_df['source'] = sorted_df['event_name']
    # добавляем целевые узлы
    sorted_df['target'] = sorted_df.groupby('session_id')['source'].shift(-1)
    
    # возврат таблицы без имени событий
    return sorted_df.drop(['event_name'], axis=1)


def get_source_index(df):
    
    """Функция генерации индексов source

    Args:
        df (pd.DataFrame): исходная таблица с признаками step, source, target.
    Returns:
        dict: словарь с индексами, именами и соответствиями индексов именам source.
    """
    
    res_dict = {}
    
    count = 0
    # получаем индексы источников
    for no, step in enumerate(df['step'].unique().tolist()):
        # получаем уникальные наименования для шага
        res_dict[no+1] = {}
        res_dict[no+1]['sources'] = df[df['step'] == step]['source'].unique().tolist()
        res_dict[no+1]['sources_index'] = []
        for i in range(len(res_dict[no+1]['sources'])):
            res_dict[no+1]['sources_index'].append(count)
            count += 1
            
    # соединим списки
    for key in res_dict:
        res_dict[key]['sources_dict'] = {}
        for name, no in zip(res_dict[key]['sources'], res_dict[key]['sources_index']):
            res_dict[key]['sources_dict'][name] = no
    return res_dict


def show_example(step, source_indexes): # =source_indexes):

    """Функция для вывода данных для конкретного шага будущей диаграммы

    Args:
        step (int): шаг.
        source_indexes (dict): словарь с данными по source на каждом шаге диаграммы
    Returns:

    """

    print(f'Пример подготовленных данных для шага {step}\n')
    for key in source_indexes[step]:
        print(f'{key}\n', source_indexes[step][key], '\n')


# Функция случайной генерации цветов
def generate_random_color():
    
    """Случайная генерация цветов rgba

    Args:
        
    Returns:
        str: Строка со сгенерированными параметрами цвета
    """
    
    # сгенерим значение для каждого канала
    r, g, b = np.random.randint(255, size=3)
    return f'rgba({r}, {g}, {b}, 1)'


def colors_for_sources(df_comp, mode):
    
    """Генерация цветов rgba

    Args:
        mode (str): сгенерировать случайные цвета, если 'random', а если 'custom' - 
                    использовать заранее подготовленные
    Returns:
        dict: словарь с цветами, соответствующими каждому индексу
    """
    # словарь, в который сложим цвета в соответствии с индексом
    colors_dict = {}
    
    if mode == 'random':
        # генерим случайные цвета
        for label in df_comp['source'].unique():
            r, g, b = np.random.randint(255, size=3)            
            colors_dict[label] = f'rgba({r}, {g}, {b}, 1)'
            
    elif mode == 'custom':
        # присваиваем ранее подготовленные цвета
        colors = requests.get('https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/json/colors_senkey.json').json()
        for no, label in enumerate(df_comp['source'].unique()):
            colors_dict[label] = colors['custom_colors'][no]
            
    return colors_dict


# Расчет количества уникальных пользователей в процентах
def percent_users(sources, targets, values):
    
    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)
    
    Args:
        sources (list): список с индексами source.
        targets (list): список с индексами target.
        values (list): список с "объемами" потоков.
        
    Returns:
        list: список с "объемами" потоков в процентах
    """
    
    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))
    
    new_list = []
    
    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}
    
    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl
                    
    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))
    
    return new_list


# def lists_for_plot(source_indexes=source_indexes, colors=colors_dict, frac=10):
def lists_for_plot(df_comp, source_indexes, colors, frac=10):
    
    """
    Создаем необходимые для отрисовки диаграммы переменные списков и возвращаем
    их в виде словаря
    
    Args:
        source_indexes (dict): словарь с именами и индексами source.
        colors (dict): словарь с цветами source.
        frac (int): ограничение на минимальный "объем" между узлами.
        
    Returns:
        dict: словарь со списками, необходимыми для диаграммы.
    """
    
    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []

    # проходим по каждому шагу
    for step in tqdm(sorted(df_comp['step'].unique()), desc='Шаг'):
        if step + 1 not in source_indexes:
            continue

        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']

        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in temp_dict_source.items(): # tqdm(temp_dict_source.items()):
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id            
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source'] == source)&(df_comp['target'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))
                    
    labels = []
    colors_labels = []
    for key in source_indexes:
        for name in source_indexes[key]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])
            
    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)
    
    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")
    
    # возвратим словарь с вложенными списками
    return {'sources': sources, 
            'targets': targets, 
            'values': values, 
            'labels': labels, 
            'colors_labels': colors_labels, 
            'link_color': link_color, 
            'link_text': link_text}


def plot_sankey_diagram(data_dict): #=data_for_plot):    
    
    """
    Функция для генерации объекта диаграммы Санкей 
    
    Args:
        data_dict (dict): словарь со списками данных для построения.
        
    Returns:
        plotly.graph_objs._figure.Figure: объект изображения.
    """
    
    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'],
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Диаграмма Санкей", font_size=10, width=1000, height=1200)
    
    # возвращаем объект диаграммы
    return fig



Мои ф-ции:

In [4]:
# ========================================================== Мои ф-ции ==============================================================================

# TODO: можно сделать создание запроса в цикле в зависимости от количества названий событий
def get_funnel_data(event_names, events):
    
    q = ''
    if len(event_names) == 2:
        q = f'''SELECT count(distinct d1.user_id) AS {event_names[0]}, 
                    count(distinct d2.user_id) AS {event_names[1]}                     
            FROM {events} d1            
            LEFT JOIN {events} d2
                 ON d2.user_id = d1.user_id
                    AND d2.session_id = d1.session_id
                    AND d2.event_time > d1.event_time
                    AND d2.event_name = '{event_names[1]}'
            WHERE d1.event_name = '{event_names[0]}' ; '''
    elif len(event_names) == 3:
        q = f'''SELECT count(distinct d1.user_id) AS {event_names[0]}, 
                    count(distinct d2.user_id) AS {event_names[1]}, 
                    count(distinct d3.user_id) AS {event_names[2]}
            FROM {events} d1
            LEFT JOIN {events} d2
                 ON d2.user_id = d1.user_id
                    AND d2.session_id = d1.session_id
                    AND d2.event_time > d1.event_time
                    AND d2.event_name = '{event_names[1]}'
            LEFT JOIN {events} d3
                 ON d3.user_id = d2.user_id
                    AND d3.session_id = d2.session_id
                    AND d3.event_time > d2.event_time
                    AND d3.event_name = '{event_names[2]}'
            WHERE d1.event_name = '{event_names[0]}' ; '''
            
    results = duckdb.sql(q).df()
    return pd.DataFrame({  'values': results.iloc[0].values, 'events': event_names})



In [5]:
# ! создание воронки без учета сессий (заменил на ф-цию с сессиями, не использую)
def get_funnel_data_dirty(event_names, events):
    
    i = 0
    users = []
    data = dict( values=[], events=[] )
    for event_name in event_names:
        if i == 0:
            users = events[ events['event.name'] == event_name ]['user.id'].unique()            
        else:
            users = events[ events['user.id'].isin(users) & (events['event.name'] == event_name) ] ['user.id'].unique()
        
        # print(f"Событие: {event_name} Всего пользователей: {len(users)}")
        
        data['values'].append(len(users))
        data['events'].append(event_name)
        i+=1
    return data


# Загрузка данных

In [6]:
url = 'https://code.s3.yandex.net/datasets/'

sources, events  = (
    pd.read_csv(f'{url}mobile_sources.csv'), # источники
    pd.read_csv(f'{url}mobile_dataset.csv', parse_dates=['event.time'], dayfirst=False), # события
)


# Первичный осмотр данных

In [7]:
ProfileReport(sources, progress_bar=False, title='Датафрейм sources').to_notebook_iframe()

In [8]:
ProfileReport(events, progress_bar=False, title='Датафрейм events').to_notebook_iframe()

Выведем все значения event.name в датафрейме events:

In [9]:
events['event.name'].value_counts(normalize=True)

tips_show        0.54
photos_show      0.13
advert_open      0.08
contacts_show    0.06
map              0.05
search_1         0.05
favorites_add    0.02
search_5         0.01
tips_click       0.01
search_4         0.01
contacts_call    0.01
search_3         0.01
search_6         0.01
search_2         0.00
search_7         0.00
show_contacts    0.00
Name: event.name, dtype: float64

**Промежуточные итоги:**

Датасет source:
* Пропущенные значения в столбцах: 0%
* Дубликаты полные: 0%
* Категориальный столбец source содержит следующие значения: yandex(45.1%), other(28.7%), google(26.3%)

Датасет events:
* Точки в названиях столбцов
* Пропущенные значения в столбцах: 0%
* Дубликаты полные: 0%
* Категориальный столбец event.time содержит даты с 2019-10-07 по 2019-11-03 
* Категориальный столбец event.name содержит следующие значения: tips_show(54%), photos_show(13.5%), advert_open(8.3%), contacts_show(6.0%), map(5.2%), search_1(4.7%), favorites_add(1.9%), search_5(1.4%), tips_click(1.1%), search_4(0.9%), другие значения (6)(2.9%)
* Категориальный столбец event.name содержит значения contacts_show и show_contacts 

**Вывод:**
* Датасет source:
    * Обработка пропусков/дубликатов не требуется
    * Категориальный столбец source содержит следующие значения: yandex(45.1%), other(28.7%), google(26.3%)
* Датасет events:
    * Необходимо заменить . в названиях столбцов на _
    * Обработка пропусков/дубликатов не требуется
    * Категориальный столбец event.time содержит даты с 2019-10-07 по 2019-11-03 => подтверждает описание, что в датасете содержатся данные пользователей, впервые совершивших действия в приложении после 7 октября 2019 года.
    * Категориальный столбец event.name содержит следующие значения: tips_show(54%), photos_show(13.5%), advert_open(8.3%), contacts_show(6.0%), map(5.2%), search_1(4.7%), favorites_add(1.9%), search_5(1.4%), tips_click(1.1%), search_4(0.9%), другие значения (6)(2.9%)
    * Категориальный столбец event.name содержит значения contacts_show и show_contacts. Скорее всего эти значения относятся к одному и тому же событию. Просто неправильно введено название, тк значения этого столбца имеют формат меню_действие. Следовательно надо заменить в датасете все вхождения show_contacts на contacts_show



# Предобработка данных

В датасете events:

* Заменим . в названиях столбцов на _
* Заменим все вхождения show_contacts на contacts_show в столбце event.name


In [10]:
# заменим . в названиях столбцов на _
events.columns = events.columns.str.replace('.', '_')

# заменим все вхождения show_contacts на contacts_show в столбце event.name
events['event_name'] = np.where(events['event_name'] == 'show_contacts', 'contacts_show', events['event_name'])

# проверим
display(events['event_name'].value_counts(normalize=True))

tips_show        0.54
photos_show      0.13
advert_open      0.08
contacts_show    0.06
map              0.05
search_1         0.05
favorites_add    0.02
search_5         0.01
tips_click       0.01
search_4         0.01
contacts_call    0.01
search_3         0.01
search_6         0.01
search_2         0.00
search_7         0.00
Name: event_name, dtype: float64

Добавим столбец длительности событий 'duration':

In [11]:
events = events.sort_values(by=['user_id', 'event_time'])
events['duration'] = events.groupby(by='user_id')['event_time'].diff()

events.head()

,event_time,event_name,user_id,duration
805,2019-10-07 13:39:45.989359,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,NaT
806,2019-10-07 13:40:31.052909,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:00:45.063550
809,2019-10-07 13:41:05.722489,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:00:34.669580
820,2019-10-07 13:43:20.735461,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:02:15.012972
830,2019-10-07 13:45:30.917502,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:02:10.182041


Выведем диаграмму размаха длительности событий для нахождения максимальной длительности сессии

In [12]:
fig = px.box(events['duration'].astype('timedelta64[m]'), y='duration')
fig.update_layout(
    title="Диаграмма размаха длительности событий",
    yaxis_title="Длительность (минуты)",
    yaxis_range=[0,10],
    width=600
)
fig.show()

Найдем максимальную длительность сессии

In [13]:
q1 = events['duration'].quantile(0.25)
q3 = events['duration'].quantile(0.75)
iqr = q3 - q1 
max_duration = q3 + 1.5 * iqr
print('Макс длительность:', max_duration)


Макс длительность: 0 days 00:06:55.359879


Из диаграммы видно, что максимальная длительность имеет значение 06:55 (~7 минут). Следовательно примем это значение за тайм-аут сессии. Разделим события на сессии по тайм-ауту сессии

In [14]:
# определим разницу в 7 минут для каждой группы с накопительной суммой
g = (events.groupby('user_id')['event_time'].diff() > pd.Timedelta('7Min')).cumsum()
# создадим счетчик групп
events['session_id'] = events.groupby(['user_id', g], sort=False).ngroup() + 1
events.head()

,event_time,event_name,user_id,duration,session_id
805,2019-10-07 13:39:45.989359,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,NaT,1
806,2019-10-07 13:40:31.052909,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:00:45.063550,1
809,2019-10-07 13:41:05.722489,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:00:34.669580,1
820,2019-10-07 13:43:20.735461,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:02:15.012972,1
830,2019-10-07 13:45:30.917502,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,0 days 00:02:10.182041,1


# Customer Journey Map (CJM)

Для визуализации пользовательского опыта выведем диаграмму Санкей. Предварительно удалим повторяющиеся события

In [15]:
# удалим повторяющиеся события
events_filtered = events.drop_duplicates(subset=['event_name','session_id'])

events.query('session_id in [7,8,9]').head(20)

,event_time,event_name,user_id,duration,session_id
33482,2019-10-20 18:49:24.115634,search_1,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 20:49:29.478536,7
33498,2019-10-20 18:59:22.541082,photos_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:09:58.425448,8
33510,2019-10-20 19:03:02.030004,favorites_add,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:03:39.488922,8
33514,2019-10-20 19:04:16.149734,search_1,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:01:14.119730,8
33523,2019-10-20 19:09:56.162564,search_1,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:05:40.012830,8
33528,2019-10-20 19:11:47.344296,search_1,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:01:51.181732,8
33533,2019-10-20 19:17:18.659799,contacts_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:05:31.315503,8
33534,2019-10-20 19:17:24.887762,contacts_call,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:00:06.227963,8
33537,2019-10-20 19:18:54.738758,photos_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:01:29.850996,8
33540,2019-10-20 19:20:41.699609,photos_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:01:46.960851,8


In [16]:
# проверка для 8й сессии
events_filtered.query('session_id == 8')

,event_time,event_name,user_id,duration,session_id
33498,2019-10-20 18:59:22.541082,photos_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:09:58.425448,8
33510,2019-10-20 19:03:02.030004,favorites_add,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:03:39.488922,8
33514,2019-10-20 19:04:16.149734,search_1,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:01:14.119730,8
33533,2019-10-20 19:17:18.659799,contacts_show,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:05:31.315503,8
33534,2019-10-20 19:17:24.887762,contacts_call,00157779-810c-4498-9e05-a1e9e3cedf93,0 days 00:00:06.227963,8


In [17]:
# преобразуем таблицу
df = add_features(events_filtered)

# ограничение количества шагов до 7 (не использую)
# удалим все пары source-target, шаг которых превышает 7 и сохраним полученную таблицу в отдельную переменную
# df_comp = df[df['step'] <= 7].copy().reset_index(drop=True)

# создаем словарь с индексами source
source_indexes = get_source_index(df)

# пример данных словаря с индексами source (шаг=2)
# show_example(2, source_indexes)  

# создаем словарь цветов с соответствиями source: color
colors_dict = colors_for_sources(df, mode='random')

# создаем словарь с данными для отрисовки диаграммы
data_for_plot = lists_for_plot(df, source_indexes, colors_dict, frac=10)

# создание объекта диаграммы
plot_sankey_diagram(data_for_plot).show()


Шаг:   0%|          | 0/9 [00:00<?, ?it/s]

Выведем воронки событий на график

In [18]:
fig = make_subplots(rows=3, cols=2, row_heights=[0.4, 0.2, 0.4]) 

data = get_funnel_data(['map', 'tips_show', 'contacts_show'], 'events_filtered')
funnel = go.Funnel(x=data['values'], y=data['events'], textposition = "inside", textinfo = "value+percent initial",)
fig.add_trace(funnel, row=1, col=1)

data = get_funnel_data(['advert_open', 'tips_show', 'contacts_show'], 'events_filtered')
funnel = go.Funnel(x=data['values'], y=data['events'], textposition = "inside", textinfo = "value+percent initial",)
fig.add_trace(funnel, row=1, col=2)

data = get_funnel_data(['tips_show', 'contacts_show'], 'events_filtered')
funnel = go.Funnel(x=data['values'], y=data['events'], textposition = "inside", textinfo = "value+percent initial",)
fig.add_trace(funnel, row=2, col=1)

data = get_funnel_data(['photos_show', 'contacts_show'], 'events_filtered')
funnel = go.Funnel(x=data['values'], y=data['events'], textposition = "inside", textinfo = "value+percent initial",)
fig.add_trace(funnel, row=2, col=2)

data = get_funnel_data(['search_1', 'photos_show', 'contacts_show'], 'events_filtered')
funnel = go.Funnel(x=data['values'], y=data['events'], textposition = "inside", textinfo = "value+percent initial",)
fig.add_trace(funnel, row=3, col=1)

fig.update_layout(height=700, width=1000, title_text="Воронки событий", showlegend=False)
fig.update_yaxes(title_text="Событие", row=1, col=1)
fig.update_yaxes(title_text="Событие", row=2, col=1)
fig.update_yaxes(title_text="Событие", row=3, col=1)

fig.show()

**Вывод:**
* Наиболее популярные сценарии перехода пользователей к целевому событию - просмотр номера телефона (contacts_show):
  * Открыл карту объявлений (map) или открыл карточки объявления (advert_open) => увидел рекомендованные объявления (tips_show) => contacts_show
  * Увидел рекомендованные объявления (tips_show) или посмотрел фотографии в объявлении (photos_show) => contacts_show
  * Выполнил поиск по сайту (search_1) => посмотрел фотографии в объявлении (photos_show) => contacts_show
* Наибольшие потоки пользователей приходят из событий открытия карты объявлений (map), через рекомендованные объявления (tips_show) и поиск по сайту (search_1). Возможно стоит сократить функционал поиска в связи с избыточностью. 

# Оценим, какие действия чаще совершают те пользователи, которые просматривают контакты

Рассчитаем относительную частоту событий в разрезе двух групп пользователей: 
* группа 1 - пользователи, которые смотрели контакты *contacts_show*
* группа 2 - пользователи, которые не смотрели контакты *contacts_show*

In [19]:
q = """
WITH group_1 AS (
        SELECT event_name, 
                ( COUNT(event_name)*1.0 / SUM(COUNT(event_name)) OVER () ) AS frequency_group_1,
        FROM events
        WHERE user_id IN (SELECT DISTINCT(user_id) 
                            FROM events
                            WHERE event_name = 'contacts_show')
        GROUP BY event_name
        ORDER BY frequency_group_1 DESC)
SELECT * 
FROM group_1
LEFT JOIN
        (SELECT event_name, 
                ( COUNT(event_name)*1.0 / SUM(COUNT(event_name)) OVER () ) AS frequency_group_2,
        FROM events
        WHERE user_id NOT IN (SELECT DISTINCT(user_id) 
                            FROM events
                            WHERE event_name = 'contacts_show')                
        GROUP BY event_name
        ORDER BY frequency_group_2 DESC) AS group_2 USING (event_name)
ORDER BY frequency_group_1 DESC
"""
print('Действия пользователей:')
results = duckdb.sql(q).df()
results.style.format('{:.1%}', subset=['frequency_group_1','frequency_group_2'], na_rep='')

Действия пользователей:


,event_name,frequency_group_1,frequency_group_2
0,tips_show,46.9%,58.1%
1,contacts_show,16.7%,
2,photos_show,14.1%,13.2%
3,advert_open,5.8%,9.7%
4,search_1,4.9%,4.6%
5,map,4.0%,5.9%
6,contacts_call,2.0%,
7,favorites_add,1.6%,2.1%
8,tips_click,1.2%,1.0%
9,search_5,0.9%,1.7%


Выведем на график относительную частоту действий

In [20]:
results = results.sort_values(by='frequency_group_1')

fig = px.bar(results, y='event_name', x=results.columns, orientation='h', barmode='group') 
# добавим подписи на диаграмму
fig.update_layout(
    title="Относительная частота действий пользователей",
    xaxis_title="Относительная частота",
    yaxis_title="Действие пользователя",
    legend_title='Группа',
    width=800,
    height=600
)
fig.show()

**Вывод:**
* Пользователи, которые не смотрели контакты не могут выполнить звонок по номеру из объявления (contacts_call)
* Частота событий пользователей, которые смотрели контакты ненамного отличается от частоты событий другой группы. Сложно сделать однозначный вывод

# Аудиторные метрики

## DAU

In [21]:
q = """ 
SELECT event_time::date AS date, COUNT(DISTINCT user_id) AS unique_users 
FROM events 
GROUP BY date
ORDER BY date;  """
results = duckdb.sql(q).df()

fig = px.line(results, x='date', y='unique_users') 
fig.update_layout(
    title="Количество уникальных пользователей по дням (DAU)",
    xaxis_title="Дата",
    yaxis_title="Количество",
    legend_title='',
    width=800,
    height=600
)
fig.show()

**Вывод:**
* Количество уникальных пользователей по дням имеет тенденцию роста. Есть резкие провалы с 10 по 12 октября и с 31 октября по 2 ноября

## WAU

In [22]:
q = """ 
SELECT EXTRACT(WEEK FROM event_time) AS week, COUNT(DISTINCT user_id) AS unique_users 
FROM events 
GROUP BY week
ORDER BY week;  """
results = duckdb.sql(q).df()

fig = px.line(results, x='week', y='unique_users') 
fig.update_layout(
    title="Количество уникальных пользователей по неделям (WAU)",
    xaxis_title="Неделя",
    yaxis_title="Количество",
    legend_title='',
    width=800,
    height=600
)
fig.show()

**Вывод:**
* На 43 неделе наблюдается резкое падение количества уникальных пользователей после роста 

# Метрики удовлетворенности (когортный анализ)

## Retention Rate по дням

In [23]:
# расчет Retention Rate по дням (start_date - когорта по дням)
q = """ 
SELECT  dt, COUNT(DISTINCT user_id) AS active_users,
        COUNT(DISTINCT user_id)::FLOAT / MAX(COUNT(DISTINCT user_id)) OVER (PARTITION BY start_date) AS retention_rate,
        start_date,
        -- DATE_TRUNC('week', start_date) AS start_week, 
        -- DATE_TRUNC('week', dt) AS week,
        dt - start_date AS dt_diff
FROM
(SELECT user_id, 
        min(event_time::date) over (partition by user_id) AS start_date, 
        event_time::date AS dt
FROM events) t1 
GROUP BY start_date, dt
ORDER BY start_date, dt;
"""
retention = duckdb.sql(q).df()
retention['start_date'] = pd.to_datetime(retention['start_date']).dt.date

# проверка
display(retention.head(10))

,dt,active_users,retention_rate,start_date,dt_diff
0,2019-10-07,204,1.00,2019-10-07,0
1,2019-10-08,37,0.18,2019-10-07,1
2,2019-10-09,21,0.10,2019-10-07,2
3,2019-10-10,22,0.11,2019-10-07,3
4,2019-10-11,14,0.07,2019-10-07,4
5,2019-10-12,12,0.06,2019-10-07,5
6,2019-10-13,8,0.04,2019-10-07,6
7,2019-10-14,16,0.08,2019-10-07,7
8,2019-10-15,13,0.06,2019-10-07,8
9,2019-10-16,10,0.05,2019-10-07,9


In [24]:

report = retention.pivot_table(index=['start_date'], columns='dt_diff', values='retention_rate').drop(columns=[0])   # удаляем 0-й день 

fig = px.imshow(report, text_auto='.1%')
fig.update_layout(
    title="Тепловая карта удержания",
    xaxis_title="Лайфтайм",
    yaxis_title="Дата когорты",
    height=800,
)
fig.show()

report.index = pd.to_datetime(report.index).date.astype('str')
report = report.T.reset_index()

fig = px.line(report, x='dt_diff', y=report.columns) 
fig.update_layout(
    title="Кривые удержания по дням привлечения",
    xaxis_title="Лайфтайм",
    yaxis_title="Процент пользователей",
    legend_title='Когорта',
    height=800,
)
fig.show()



**Вывод:**
* Сложно выделить какие-то определенные когорты на основании тепловой карты удержания
* В когортах от 11.10 и 13.10 есть дни когда пользователи когорты отсутствовали в приложении. Возможно это ошибка данных
* Кривые удержания содержат резкие скачки количества пользователей. Возможно это связано с пуш-уведомлениями приложения или рекламой

## Retention Rate по неделям

In [25]:
# расчет Retention Rate по неделям (start_week - когорта по неделям)
q = """ 
SELECT  week, COUNT(DISTINCT user_id) AS active_users,
        COUNT(DISTINCT user_id)::FLOAT / MAX(COUNT(DISTINCT user_id)) OVER (PARTITION BY start_week) AS retention_rate,
        start_week, 
        week - start_week AS week_diff
FROM
(SELECT user_id, 
        min(EXTRACT(WEEK FROM event_time)) over (partition by user_id) AS start_week, 
        EXTRACT(WEEK FROM event_time) AS week
FROM events) t1 
GROUP BY start_week, week
ORDER BY start_week, week;
"""
retention = duckdb.sql(q).df()

# проверка
display(retention.head(10))

,week,active_users,retention_rate,start_week,week_diff
0,41,1130,1.00,41,0
1,42,272,0.24,41,1
2,43,170,0.15,41,2
3,44,119,0.11,41,3
4,42,1166,1.00,42,0
5,43,282,0.24,42,1
6,44,155,0.13,42,2
7,43,1094,1.00,43,0
8,44,239,0.22,43,1
9,44,903,1.00,44,0


In [26]:
report = retention.pivot_table(index=['start_week'], columns='week_diff', values='retention_rate').drop(columns=[0])   # удаляем 0-ю неделю 

fig = px.imshow(report, text_auto='.1%')
fig.update_layout(
    title="Тепловая карта удержания",
    xaxis_title="Лайфтайм",
    yaxis_title="Неделя когорты",
    width=600,
)
fig.show()

report = report.T.reset_index()

fig = px.line(report, x='week_diff', y=report.columns) 
fig.update_layout(
    title="Кривые удержания по неделям привлечения",
    xaxis_title="Лайфтайм",
    yaxis_title="Процент пользователей",
    legend_title='Когорта',
    width=600,
)
fig.show()

**Вывод:**
* По диаграммам Retention rate по неделям можно сделать предположение, что за 3 недели пользователь разочаровывается в приложении. К сожалению, имеются данные только за 4 недели. Этого периода недостаточно для точного прогноза динамики.

## Churn Rate по дням

In [27]:
# расчет Churn Rate по дням (start_date - когорта по дням)
q = """ 
SELECT *,
        LAG(cnt_users) OVER (PARTITION BY start_date ORDER BY dt) AS previous_day_cnt_users,
        1 - (cnt_users::numeric/ LAG(cnt_users) OVER (PARTITION BY start_date ORDER BY dt)) AS churn_rate,
        dt - start_date AS dt_diff
FROM
        (SELECT  start_date, dt,
                COUNT(DISTINCT user_id) AS cnt_users
        FROM
                (SELECT user_id, 
                        min(event_time::date) over (partition by user_id) AS start_date, 
                        event_time::date AS dt
                FROM events) t1 
        GROUP BY start_date, dt
        ORDER BY start_date, dt) t2
ORDER BY start_date, dt
"""
churn = duckdb.sql(q).df()
churn['start_date'] = pd.to_datetime(churn['start_date']).dt.date

# проверка
display(churn.head(10))

,start_date,dt,cnt_users,previous_day_cnt_users,churn_rate,dt_diff
0,2019-10-07,2019-10-07,204,NaN,NaN,0
1,2019-10-07,2019-10-08,37,204.00,0.82,1
2,2019-10-07,2019-10-09,21,37.00,0.43,2
3,2019-10-07,2019-10-10,22,21.00,-0.05,3
4,2019-10-07,2019-10-11,14,22.00,0.36,4
5,2019-10-07,2019-10-12,12,14.00,0.14,5
6,2019-10-07,2019-10-13,8,12.00,0.33,6
7,2019-10-07,2019-10-14,16,8.00,-1.00,7
8,2019-10-07,2019-10-15,13,16.00,0.19,8
9,2019-10-07,2019-10-16,10,13.00,0.23,9


In [28]:
report = churn.pivot_table(index=['start_date'], columns='dt_diff', values='churn_rate') 

fig = px.imshow(report, text_auto='.1%')
fig.update_layout(
    title="Тепловая карта оттока пользователей",
    xaxis_title="Лайфтайм",
    yaxis_title="Дата когорты",
    height=800,
)
fig.show()

report.index = pd.to_datetime(report.index).date.astype('str')
report = report.T.reset_index()

fig = px.line(report, x='dt_diff', y=report.columns) 
fig.update_layout(
    title="Кривые оттока пользователей по дням привлечения",
    xaxis_title="Лайфтайм",
    yaxis_title="Процент пользователей",
    legend_title='Когорта',
    height=800,
)
fig.show()

## Conversion rate. Конверсия в просмотры контактов 'contacts_show'

In [29]:
# расчет конверсии в просмотры контактов 'contacts_show'
q = """
SELECT  start_date, dt,
        converted_users, users_firstday, SUM(converted_users) OVER (partition by start_date rows unbounded preceding) as cumsum,
        SUM(converted_users) OVER (partition by start_date rows unbounded preceding) / users_firstday AS conversion_rate,
        dt - start_date AS dt_diff
        -- DATE_TRUNC('week', start_date) AS start_week, 
        -- DATE_TRUNC('week', dt) AS week,
FROM 
    (SELECT  distinct dt, start_date, 
            COUNT(DISTINCT user_id) FILTER (WHERE dt = dt_conversion) OVER (partition by start_date, dt) AS converted_users,        
            COUNT(DISTINCT user_id) OVER (partition by start_date) AS users_firstday,                
    FROM
        (SELECT  distinct(user_id) AS user_id, -- уникальные пользователи по дням
                min(event_time::date) over (partition by user_id) AS start_date, -- дата когорты
                event_time::date AS dt, -- дата события
                min(event_time::date) FILTER (WHERE event_name = 'contacts_show') over (partition by user_id) AS dt_conversion, -- дата конверсии
        FROM events
        GROUP BY event_time, dt, user_id, event_name
        ORDER BY start_date, dt, user_id) t1
    ORDER BY start_date, dt) t2
ORDER BY start_date, dt
"""
conversion = duckdb.sql(q).df()
conversion['start_date'] = pd.to_datetime(conversion['start_date']).dt.date
conversion['dt'] = pd.to_datetime(conversion['dt']).dt.date

# проверка
display(conversion.head(10))

,start_date,dt,converted_users,users_firstday,cumsum,conversion_rate,dt_diff
0,2019-10-07,2019-10-07,34,204,34.00,0.17,0
1,2019-10-07,2019-10-08,4,204,38.00,0.19,1
2,2019-10-07,2019-10-09,0,204,38.00,0.19,2
3,2019-10-07,2019-10-10,1,204,39.00,0.19,3
4,2019-10-07,2019-10-11,0,204,39.00,0.19,4
5,2019-10-07,2019-10-12,1,204,40.00,0.20,5
6,2019-10-07,2019-10-13,1,204,41.00,0.20,6
7,2019-10-07,2019-10-14,1,204,42.00,0.21,7
8,2019-10-07,2019-10-15,1,204,43.00,0.21,8
9,2019-10-07,2019-10-16,1,204,44.00,0.22,9


In [30]:
# сводная таблица
report = conversion.pivot_table(index=['start_date'], columns='dt_diff', values='conversion_rate') 

fig = px.imshow(report, text_auto='.1%')
fig.update_layout(
    title="Тепловая карта конверсии",
    xaxis_title="Лайфтайм",
    yaxis_title="Дата когорты",
    height=800,
)
fig.show()

report.index = pd.to_datetime(report.index).date.astype('str')
report = report.T.reset_index()

fig = px.line(report, x='dt_diff', y=report.columns) 
fig.update_layout(
    title="Кривые конверсии по дням привлечения",
    xaxis_title="Лайфтайм",
    yaxis_title="Процент пользователей",
    legend_title='Когорта',
    height=800,
)
fig.show()

**Вывод:**
* Наилучшая конверсия в просмотры наблюдается в когорте от 09.10.2019
* Наихудшая конверсия в просмотры наблюдается в когорте от 11.10.2019
* В когортах от 11.10 и 13.10 есть дни когда пользователи когорты отсутствовали в приложении. Возможно это ошибка данных

# AB тесты

Проверим статистические гипотезы с помощью AB тестов.

## Гипотеза: конверсия в просмотры контактов различается в группах, где в первой одни пользователи совершают действия  tips_show  и  tips_click , а во второй только  tips_show 

Сформулируем гипотезы:
* **Нулевая гипотеза H0:** Нет статистически значимой разницы в конверсии в просмотры контактов у этих двух групп
* **Альтернативная H1:** Присутствует статистически значимая разница в конверсии в просмотры контактов у этих двух групп  

Пороговое значение уровня стат значимости alpha примем за 5%.      

Для проверки гипотезы проверим, какое количество пользователей совершило целевое действие:

In [31]:
# пользователи с tips_show и c tips_click (без учета сессии и последовательности)
# рассчитаем конверсию в просмотры контактов 'contacts_show' по датафрейму без повторяющихся событий events_filtered

q = """ 
SELECT  COUNT(distinct user_id) FILTER (WHERE event_name = 'contacts_show') contacts_show,
        COUNT(distinct user_id) AS total_users
FROM events_filtered
WHERE user_id IN
    -- уникальные пользователи у которых есть tips_show и c tips_click
    (SELECT distinct user_id 
    FROM events_filtered e
    WHERE event_name = 'tips_show' 
        AND user_id IN
            (SELECT distinct user_id
            FROM events_filtered 
            WHERE event_name = 'tips_click') )
"""
group_a = duckdb.sql(q).df()
group_a


,contacts_show,total_users
0,91,297


In [32]:
# пользователи с tips_show и без tips_click (без учета сессии и последовательности)
# рассчитаем конверсию в просмотры контактов 'contacts_show' по датафрейму без повторяющихся событий events_filtered

q = """ 
SELECT  COUNT(distinct user_id) FILTER (WHERE event_name = 'contacts_show') contacts_show,
        COUNT(distinct user_id) AS total_users
FROM events_filtered
WHERE user_id IN
    -- уникальные пользователи у которых есть tips_show и нет tips_click
    (SELECT distinct user_id 
    FROM events_filtered e
    WHERE event_name = 'tips_show' 
        AND user_id NOT IN
            (SELECT distinct user_id
            FROM events_filtered 
            WHERE event_name = 'tips_click') )
"""
group_b = duckdb.sql(q).df()
group_b

,contacts_show,total_users
0,425,2504


Проведем статистическое сравнение долей:

In [33]:
# статистическое сравнение долей 

alpha = 0.05

stat, p_value = proportions_ztest(np.array([group_a.loc[0,'contacts_show'], group_b.loc[0,'contacts_show']]),
                                np.array([group_a.loc[0,'total_users'], group_b.loc[0,'total_users']]))
print("p-value: ", p_value)

if (p_value < alpha): 
        print("Отвергаем нулевую гипотезу, между выборками есть статистически значимые различия.")
else:
        print("Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет.")

p-value:  9.218316568768822e-09
Отвергаем нулевую гипотезу, между выборками есть статистически значимые различия.


**Вывод:**
* Одни пользователи совершают действия  tips_show  и  tips_click , другие — только  tips_show . Проверили гипотезу: конверсия в просмотры контактов различается у этих двух групп. Между выборками есть статистически значимые различия.

## Гипотеза: Пользователи одной группы совершают действие map и затем tips_show, другой map. Конверсия в просмотры контактов различается у этих двух групп.

Проверим собственную гипотезу исходя из данных воронок событий. Сформулируем гипотезы:
* **Нулевая гипотеза:** Нет статистически значимой разницы в конверсии в просмотры контактов у этих двух групп
* **Альтернативная:** Присутствует статистически значимая разница в конверсии в просмотры контактов у этих двух групп 

Пороговое значение уровня стат значимости alpha примем за 5%.      

Для проверки гипотезы проверим, какое количество пользователей совершило целевое действие

In [34]:
# map -> tips_show -> contacts_show (с учетом сессии и последовательности)
q = '''SELECT   count(distinct d1.user_id) AS map, 
                count(distinct d2.user_id) AS total_users, 
                count(distinct d3.user_id) AS contacts_show
        FROM events_filtered d1
        LEFT JOIN events_filtered d2
                ON d2.user_id = d1.user_id
                AND d2.session_id = d1.session_id
                AND d2.event_time > d1.event_time
                AND d2.event_name = 'tips_show'
        LEFT JOIN events_filtered d3
                ON d3.user_id = d2.user_id
                AND d3.session_id = d2.session_id
                AND d3.event_time > d2.event_time
                AND d3.event_name = 'contacts_show'
        WHERE d1.event_name = 'map' ; '''
group_a = duckdb.sql(q).df()
group_a

,map,total_users,contacts_show
0,1456,985,102


In [35]:
# map -> contacts_show (с учетом сессии и последовательности)
# исключаем события tips_show
q = '''SELECT   count(distinct d1.user_id) AS total_users, 
                count(distinct d2.user_id) AS contacts_show 
        FROM events_filtered d1
        LEFT JOIN events_filtered d2
                ON d2.user_id = d1.user_id
                AND d2.session_id = d1.session_id
                AND d2.event_time > d1.event_time
                AND d2.event_name = 'contacts_show'
        WHERE d1.event_name = 'map' 
                AND d1.user_id NOT IN 
                        (SELECT distinct user_id
                        FROM events_filtered 
                        WHERE event_name = 'tips_show'
                        )

        '''
group_b = duckdb.sql(q).df()
group_b

,total_users,contacts_show
0,104,7


Проведем статистическое сравнение долей:

In [36]:
# статистическое сравнение долей 

alpha = 0.05

stat, p_value = proportions_ztest(np.array([group_a.loc[0,'contacts_show'], group_b.loc[0,'contacts_show']]),
                                np.array([group_a.loc[0,'total_users'], group_b.loc[0,'total_users']]))
print("p-value: ", p_value)

if (p_value < alpha): 
        print("Отвергаем нулевую гипотезу, между выборками есть статистически значимые различия.")
else:
        print("Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет.")

p-value:  0.2414677276286613
Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет.


**Вывод:**
* Пользователи одной группы совершают действие map и затем tips_show, другой map. Проверили гипотезу: конверсия в просмотры контактов различается у этих двух групп. Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет.

# Общие выводы и рекомендации для улучшения продуктовых метрик приложения

* Наиболее популярные сценарии перехода пользователя к целевому событию - просмотру номера телефона (contacts_show):
  * Открыл карту объявлений (map) или открыл карточки объявления (advert_open) => увидел рекомендованные объявления (tips_show) => contacts_show
  * Увидел рекомендованные объявления (tips_show) или посмотрел фотографии в объявлении (photos_show) => contacts_show
  * Выполнил поиск по сайту (search_1) => посмотрел фотографии в объявлении (photos_show) => contacts_show
* Наибольшие потоки пользователей приходят из событий открытия карты объявлений (map), через рекомендованные объявления (tips_show) и поиск по сайту (search_1). Возможно стоит сократить функционал поиска в связи с избыточностью. 
* Количество уникальных пользователей по дням имеет тенденцию роста. Есть резкие провалы с 10 по 12 октября и с 31 октября по 2 ноября
* На 43 неделе наблюдается резкое падение количества уникальных пользователей после роста 
* По диаграммам Retention rate по неделям можно сделать предположение, что за 3 недели пользователь разочаровывается в приложении. К сожалению, имеются данные только за 4 недели. Этого периода недостаточно для точного прогноза динамики.
* Наилучшая конверсия в просмотры наблюдается в когорте от 09.10.2019
* Наихудшая конверсия в просмотры наблюдается в когорте от 11.10.2019
* В когортах от 11.10 и 13.10 есть дни когда пользователи когорты отсутствовали в приложении. Возможно это ошибка данных
* Одни пользователи совершают действия  tips_show  и  tips_click , другие — только  tips_show . Проверили гипотезу: конверсия в просмотры контактов различается у этих двух групп. Между выборками есть статистически значимые различия.
* Пользователи одной группы совершают действие map и затем tips_show, другой map. Проверили гипотезу: конверсия в просмотры контактов различается у этих двух групп. Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет.